#### Step 1: Import the dataset and set the minSupport and minConfidence

[REFERENCE](https://hands-on.cloud/implementation-of-fp-growth-algorithm-using-python/)

In [244]:
import pandas as pd
import numpy as np
from itertools import combinations
from csv import reader
from collections import defaultdict
from mlxtend.preprocessing import TransactionEncoder

min_support = 0.4
min_confidence = 1

# Step 0 Preprocessing
df = pd.read_csv('data.csv')
transactions = []
for i in range(len(df)):
    transaction = []
    for col in df.columns:
        if df.loc[i, col] == True:
            transaction.append(col)
    transactions.append(transaction)

# Convert the transactions into a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)


In [245]:
# Step 1: Calculate the support for each item
support_counts = {}
for transaction in transactions:
    for item in transaction:
        if item in support_counts:
            support_counts[item] += 1
        else:
            support_counts[item] = 1

print(support_counts)

{'apt': 3, 'poor': 1, 'car': 4, 'avg': 2, 'villa': 2, 'rich': 2}


In [246]:
# Step 2: Calculate F-list
# Remove items that do not meet the minimum support threshold
f_list = []
for item, support in support_counts.items():
    if support >= min_support:
        f_list.append(item)

# Sort by support
f_list = sorted(f_list, key=lambda x: support_counts[x], reverse=True)

print(f_list)

['car', 'apt', 'avg', 'villa', 'rich', 'poor']


In [260]:
# Step 4: Create Fp-Tree
class FPNode:
    def __init__(self, name, count, parent):
        self.name = name
        self.count = count
        self.parent = parent
        self.children = {}
        self.next = None

def build_FPTree(transactions, f_list, min_support):
    root = FPNode('root', 0, None)
    headerTable = {}
    
    for transaction in transactions:
        transaction = [item for item in transaction if item in f_list]
        transaction.sort(key=lambda x: f_list.index(x))
        curr_node = root
        for item in transaction:
            if item in curr_node.children:
                curr_node.children[item].count += 1
            else:
                new_node = FPNode(item, 1, curr_node)
                curr_node.children[item] = new_node
                if item in headerTable:
                    headerTable[item].append(new_node)
                else:
                    headerTable[item] = [new_node]
            curr_node = curr_node.children[item]
    
    # Convert headerTable items to lists
    for item in headerTable:
        if not isinstance(headerTable[item], list):
            headerTable[item] = [headerTable[item]]
    
    return root, headerTable


In [261]:
def get_conditional_pattern_base(item, headerTable):
    conditional_pattern_base = []
    node = headerTable[item][0].next
    while node:
        # Get the path from the node to the root
        path = []
        curr_node = node
        while curr_node.parent.name != 'root':
            path.append(curr_node.parent.name)
            curr_node = curr_node.parent

        # Add the path to the conditional pattern base with the count of the node
        conditional_pattern_base.append((path[::-1], node.count))
        node = node.next

    return conditional_pattern_base


In [262]:
# Step 5: Mine Fp-Tree
def mine_FPTree(root, headerTable, min_support, frequent_itemsets=[], prefix=[]):
    # Sort the items in the header table by support
    items = sorted(headerTable.keys(), key=lambda x: f_list.index(x))
    for item in items:
        # Create new prefix
        new_prefix = prefix + [item]
        # Add the new prefix to the list of frequent itemsets
        frequent_itemsets.append(new_prefix)
        # Get the conditional pattern base for the item
        conditional_pattern_base = get_conditional_pattern_base(item, headerTable)
        # Build the conditional FPTree
        conditional_FPTree, conditional_headerTable = build_FPTree(conditional_pattern_base, f_list, min_support)
        # Mine the conditional FPTree
        if conditional_headerTable:
            mine_FPTree(conditional_FPTree, conditional_headerTable, min_support, frequent_itemsets, new_prefix)

    return frequent_itemsets


In [263]:
root, headerTable = build_FPTree(transactions, f_list, min_support)
frequent_itemsets = mine_FPTree(root, headerTable, min_support)

print(frequent_itemsets)

[['car'], ['apt'], ['avg'], ['villa'], ['rich'], ['poor']]


In [255]:
# Step 6: Generate Association Rules
def generate_association_rules(frequent_itemsets, min_confidence):
    rules = []
    for itemset in frequent_itemsets:
        # Get all subsets of the itemset
        subsets = []
        for i in range(1, len(itemset)):
            subsets += combinations(itemset, i)

        # Calculate the confidence of each subset
        for subset in subsets:
            confidence = support_counts[tuple(itemset)] / support_counts[tuple(subset)]
            if confidence >= min_confidence:
                rules.append((subset, tuple(set(itemset) - set(subset)), confidence))
    return rules

rules = generate_association_rules(frequent_itemsets, min_confidence)

print(rules)

[]
